In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc).builder.config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size").enableHiveSupport().getOrCreate()

23/12/08 05:31:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
years = [i for i in range(2009, 2023)]
files = [[] for i in range(len(years)+1)]

for idx, year in enumerate(years):
    for month in range(1, 13):
        files[idx].append(f"gs://training-project-404110.appspot.com/Data/yellow_tripdata_{year}-{month:02}.parquet")

years.append(2023)
for month in range(1, 9):
    files[-1].append(f"gs://training-project-404110.appspot.com/Data/yellow_tripdata_2023-{month:02}.parquet")
print(files)



[['gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-01.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-02.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-03.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-04.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-05.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-06.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-07.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-08.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-09.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-10.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-11.parquet', 'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-12.parquet'], ['gs://training-project-4

In [4]:
files

[['gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-01.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-02.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-03.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-04.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-05.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-06.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-07.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-08.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-09.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-10.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-11.parquet',
  'gs://training-project-404110.appspot.com/Data/yellow_tripdata_2009-12.parquet'],
 ['

In [5]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, TimestampType, IntegerType

schemas = {}

schemas[2009] = StructType([
    StructField("Trip_Pickup_DateTime", StringType(), True),
    StructField("Trip_Dropoff_DateTime", StringType(), True),
    StructField("Passenger_Count", LongType(), True),
    StructField("Trip_Distance", DoubleType(), True),
    StructField("Start_Lon", DoubleType(), True),
    StructField("Start_Lat", DoubleType(), True),
    StructField("End_Lon", DoubleType(), True),
    StructField("End_Lat", DoubleType(), True),
    StructField("Payment_Type", StringType(), True),
    StructField("Fare_Amt", DoubleType(), True),
    StructField("Tip_Amt", DoubleType(), True),
    StructField("Tolls_Amt", DoubleType(), True),
    StructField("Total_Amt", DoubleType(), True),  
])

schemas[2010] = StructType([
    StructField("pickup_datetime", StringType(), True),
    StructField("dropoff_dateTime", StringType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

schemas[2017] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

schemas[2018] = {}
schemas[2018][0] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

schemas[2018][1] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])


schemas[2019] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

schemas['rest'] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

schemas[2023] = {} 
schemas[2023][0] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

schemas[2023][1] = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_dateTime", TimestampType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),  
])

In [6]:
complete_data = {}

for idx, year in enumerate(years):
    if year == 2009:
        schema = schemas[2009]
    elif year == 2010:
        schema = schemas[2010]
    elif year < 2018:
        
        schema = schemas[2017]
    elif year == 2018:
        complete_data[2018] = {}
        
        schema = schemas[2018][0]
        df = spark.read.option("mergeSchema", "true").schema(schema).parquet(*files[idx][:6])
        complete_data[year][0] = df
        
        schema = schemas[2018][1]
        df = spark.read.option("mergeSchema", "true").schema(schema).parquet(*files[idx][6:])
        complete_data[year][1] = df
        continue
        
    elif year == 2019:
        schema = schemas[2019]
    elif year < 2023:
        schema = schemas['rest']
    else:
        complete_data[2023] = {}
        
        schema = schemas[2023][0]
        df = spark.read.option("mergeSchema", "true").schema(schema).parquet(*files[idx][:1])
        complete_data[year][0] = df
        
        schema = schemas[2023][1]
        df = spark.read.option("mergeSchema", "true").schema(schema).parquet(*files[idx][1:])
        complete_data[year][1] = df
        continue
        
    df = spark.read.option("mergeSchema", "true").schema(schema).parquet(*files[idx][1:])  
    complete_data[year] = df
    
print(complete_data)

{2009: DataFrame[Trip_Pickup_DateTime: string, Trip_Dropoff_DateTime: string, Passenger_Count: bigint, Trip_Distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, Payment_Type: string, Fare_Amt: double, Tip_Amt: double, Tolls_Amt: double, Total_Amt: double], 2010: DataFrame[pickup_datetime: string, dropoff_dateTime: string, passenger_count: bigint, trip_distance: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double], 2011: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double], 2012: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_cou

In [7]:
print(complete_data.keys())

dict_keys([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])


In [8]:
complete_data[2018][1].show()

+--------------------+---------------------+---------------+-------------+------------+------------+------------+-----------+----------+------------+------------+
|tpep_pickup_datetime|tpep_dropoff_dateTime|passenger_count|trip_distance|PULocationID|DOLocationID|payment_type|fare_amount|tip_amount|tolls_amount|total_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+------------+-----------+----------+------------+------------+
| 2018-10-01 00:23:34|  2018-10-01 00:44:50|            1.0|          6.2|          68|           7|           2|       20.5|       0.0|         0.0|        21.8|
| 2018-10-01 00:40:05|  2018-10-01 01:01:56|            1.0|         12.6|         132|           9|           2|       35.0|       0.0|         0.0|        36.3|
| 2018-10-01 00:05:35|  2018-10-01 00:19:38|            1.0|          6.1|          50|         244|           1|       19.0|      5.05|         0.0|       25.35|
| 2018-10-01 00:42:56|

In [9]:
complete_data[2009].show()

+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+--------+-------+---------+---------+
|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|Passenger_Count|     Trip_Distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|Payment_Type|Fare_Amt|Tip_Amt|Tolls_Amt|Total_Amt|
+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+--------+-------+---------+---------+
| 2009-02-03 08:25:00|  2009-02-03 08:33:39|              1|               1.6|        -73.992768|40.758325|         -73.99471|40.739723|        CASH|     6.9|    0.0|      0.0|      6.9|
| 2009-02-28 00:26:00|  2009-02-28 00:40:00|              5|              3.05|               0.0|      0.0|               0.0|      0.0|        CASH|    10.5|    0.0|      0.0|     11.0|
| 2009-02-22 00:39:23|  2009-02-22 00:45:52|              1|

In [10]:
from pyspark.sql.functions import col
complete_data[2018][1] = complete_data[2023][1].withColumn("passenger_count", col("passenger_count").cast(LongType()))

new2018 = complete_data[2018][0].union(complete_data[2018][1])
complete_data[2018] = new2018

In [11]:
from pyspark.sql.functions import col
complete_data[2023][0] = complete_data[2023][0].withColumn("passenger_count", col("passenger_count").cast(LongType()))

complete_data[2023][1] = complete_data[2023][1].withColumn("DOLocationID", col("DOLocationID").cast(LongType()))
complete_data[2023][1] = complete_data[2023][1].withColumn("PULocationID", col("PULocationID").cast(LongType()))

In [12]:
new2023 = complete_data[2023][0].union(complete_data[2023][1])

In [13]:
new2023.show()

+--------------------+---------------------+---------------+-------------+------------+------------+------------+-----------+----------+------------+------------+
|tpep_pickup_datetime|tpep_dropoff_dateTime|passenger_count|trip_distance|PULocationID|DOLocationID|payment_type|fare_amount|tip_amount|tolls_amount|total_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+------------+-----------+----------+------------+------------+
| 2023-01-01 00:32:10|  2023-01-01 00:40:36|              1|         0.97|         161|         141|           2|        9.3|       0.0|         0.0|        14.3|
| 2023-01-01 00:55:08|  2023-01-01 01:01:27|              1|          1.1|          43|         237|           1|        7.9|       4.0|         0.0|        16.9|
| 2023-01-01 00:25:04|  2023-01-01 00:37:49|              1|         2.51|          48|         238|           1|       14.9|      15.0|         0.0|        34.9|
| 2023-01-01 00:03:48|

In [14]:
complete_data[2023] = new2023

In [15]:
complete_data

{2009: DataFrame[Trip_Pickup_DateTime: string, Trip_Dropoff_DateTime: string, Passenger_Count: bigint, Trip_Distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, Payment_Type: string, Fare_Amt: double, Tip_Amt: double, Tolls_Amt: double, Total_Amt: double],
 2010: DataFrame[pickup_datetime: string, dropoff_dateTime: string, passenger_count: bigint, trip_distance: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2011: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2012: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_

In [16]:
# Fix 2009

complete_data[2009] = complete_data[2009].withColumn("Trip_Pickup_DateTime", col("Trip_Pickup_DateTime").cast(TimestampType()))
complete_data[2009] = complete_data[2009].withColumnRenamed("Trip_Pickup_DateTime", "tpep_pickup_dateTime")

complete_data[2009] = complete_data[2009].withColumn("Trip_Dropoff_DateTime", col("Trip_Dropoff_DateTime").cast(TimestampType()))
complete_data[2009] = complete_data[2009].withColumnRenamed("Trip_Dropoff_DateTime", "tpep_dropoff_dateTime")

complete_data[2009] = complete_data[2009].withColumnRenamed("Passenger_Count", "passenger_count")
complete_data[2009] = complete_data[2009].withColumnRenamed("Payment_Type", "payment_type")
complete_data[2009] = complete_data[2009].withColumnRenamed("Fare_Amt", "fare_amount")
complete_data[2009] = complete_data[2009].withColumnRenamed("Tip_Amt", "tip_amount")
complete_data[2009] = complete_data[2009].withColumnRenamed("Tolls_Amt", "tolls_amount")
complete_data[2009] = complete_data[2009].withColumnRenamed("Total_Amt", "total_amount")
complete_data[2009] = complete_data[2009].withColumnRenamed("Trip_Distance", "trip_distance")

In [17]:
complete_data

{2009: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2010: DataFrame[pickup_datetime: string, dropoff_dateTime: string, passenger_count: bigint, trip_distance: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2011: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2012: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: tim

In [18]:
complete_data[2010] = complete_data[2010].withColumn("pickup_datetime", col("pickup_datetime").cast(TimestampType()))
complete_data[2010] = complete_data[2010].withColumnRenamed("pickup_datetime", "tpep_pickup_dateTime")

complete_data[2010] = complete_data[2010].withColumn("dropoff_dateTime", col("dropoff_dateTime").cast(TimestampType()))
complete_data[2010] = complete_data[2010].withColumnRenamed("dropoff_dateTime", "tpep_dropoff_dateTime")

complete_data[2010] = complete_data[2010].withColumnRenamed("pickup_longitude", "Start_Lon")
complete_data[2010] = complete_data[2010].withColumnRenamed("pickup_latitude", "Start_Lat")
complete_data[2010] = complete_data[2010].withColumnRenamed("dropoff_longitude", "End_Lon")
complete_data[2010] = complete_data[2010].withColumnRenamed("dropoff_latitude", "End_Lat")

In [19]:
complete_data

{2009: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2010: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2011: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2012: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenge

In [20]:
csv_path = "gs://training-project-404110.appspot.com/coordinates_zone_wise.csv"
df = spark.read.csv(csv_path, header=True, inferSchema=True)

In [21]:
df = df.withColumn("LocationID", col("LocationID").cast(LongType()))

In [22]:
df

DataFrame[LocationID: bigint, Borough: string, Zone: string, service_zone: string, Location: string, Latitude: double, Longitude: double]

In [23]:
payment_mapping = {1: "CREDIT", 2: "CASH", 3: "NO CHARGE", 4: "DISPUTE", 5: "UNKNOWN", 6: "VOIDED TRIP"}
 
def map_payment_type(payment_type):
    return payment_mapping.get(payment_type, "UNKNOWN")

In [24]:
from pyspark.sql.functions import when
for i in range(2011, 2024):
    complete_data[i] = complete_data[i].join(df.select('LocationID', 'Longitude','Latitude'), col('PULocationID') == col('LocationID'), how="inner")
    complete_data[i] = complete_data[i].withColumnRenamed("Longitude", "Start_Lon")
    complete_data[i] = complete_data[i].withColumnRenamed("Latitude", "Start_Lat").drop('PULocationID').drop('LocationID')

    complete_data[i] = complete_data[i].join(df.select('LocationID', 'Longitude','Latitude'), col('DOLocationID') == col('LocationID'), how="inner")
    complete_data[i] = complete_data[i].withColumnRenamed("Longitude", "End_Lon")
    complete_data[i] = complete_data[i].withColumnRenamed("Latitude", "End_Lat").drop('DOLocationID').drop('LocationID')
    
    complete_data[i] = complete_data[i].withColumn(
    "payment_type",
    when(col("payment_type") == 1, payment_mapping[1])
    .when(col("payment_type") == 2, payment_mapping[2])
    .when(col("payment_type") == 3, payment_mapping[3])
    .when(col("payment_type") == 4, payment_mapping[4])
    .when(col("payment_type") == 5, payment_mapping[5])
    .when(col("payment_type") == 6, payment_mapping[6])
    .otherwise("Unknown"))

In [25]:
complete_data

{2009: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2010: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2011: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double],
 2012: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_d

In [26]:
from pyspark.sql.functions import col, month, year

new_complete_data = complete_data[2009]
new_complete_data = new_complete_data.withColumn("Month", month(col("tpep_pickup_dateTime")))
new_complete_data = new_complete_data.withColumn("Year", year(col("tpep_pickup_dateTime"))) 
    
order = ['tpep_pickup_dateTime', 'tpep_dropoff_dateTime', 'passenger_count', 'trip_distance', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat', 'payment_type', 'fare_amount', 'tip_amount', 'tolls_amount', 'total_amount']

for years in list(complete_data.keys())[1:]:
    print(years)
    
    df = complete_data[years].select(*order)
    
    df = df.withColumn("Month", month(col("tpep_pickup_dateTime")))
    df = df.withColumn("Year", year(col("tpep_pickup_dateTime"))) 

    df = df.filter(df["Year"] == years)
    new_complete_data = new_complete_data.union(df)
    

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [27]:
complete_data

{2009: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2010: DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double],
 2011: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: bigint, trip_distance: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double],
 2012: DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_d

In [28]:
new_complete_data

DataFrame[tpep_pickup_dateTime: timestamp, tpep_dropoff_dateTime: timestamp, passenger_count: double, trip_distance: double, Start_Lon: double, Start_Lat: double, End_Lon: double, End_Lat: double, payment_type: string, fare_amount: double, tip_amount: double, tolls_amount: double, total_amount: double, Month: int, Year: int]

In [29]:
new_complete_data.show()

+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+-----------+----------+------------+------------+-----+----+
|tpep_pickup_dateTime|tpep_dropoff_dateTime|passenger_count|     trip_distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|payment_type|fare_amount|tip_amount|tolls_amount|total_amount|Month|Year|
+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+-----------+----------+------------+------------+-----+----+
| 2009-02-03 08:25:00|  2009-02-03 08:33:39|            1.0|               1.6|        -73.992768|40.758325|         -73.99471|40.739723|        CASH|        6.9|       0.0|         0.0|         6.9|    2|2009|
| 2009-02-28 00:26:00|  2009-02-28 00:40:00|            5.0|              3.05|               0.0|      0.0|               0.0|      0.0|        CASH|      

In [30]:
new_complete_data.show()

+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+-----------+----------+------------+------------+-----+----+
|tpep_pickup_dateTime|tpep_dropoff_dateTime|passenger_count|     trip_distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|payment_type|fare_amount|tip_amount|tolls_amount|total_amount|Month|Year|
+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+-----------+----------+------------+------------+-----+----+
| 2009-02-03 08:25:00|  2009-02-03 08:33:39|            1.0|               1.6|        -73.992768|40.758325|         -73.99471|40.739723|        CASH|        6.9|       0.0|         0.0|         6.9|    2|2009|
| 2009-02-28 00:26:00|  2009-02-28 00:40:00|            5.0|              3.05|               0.0|      0.0|               0.0|      0.0|        CASH|      

In [31]:
from pyspark.sql.functions import col, upper
new_complete_data = new_complete_data.withColumn("payment_type", upper(col("payment_type")))

In [32]:
new_complete_data.show()

+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+-----------+----------+------------+------------+-----+----+
|tpep_pickup_dateTime|tpep_dropoff_dateTime|passenger_count|     trip_distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|payment_type|fare_amount|tip_amount|tolls_amount|total_amount|Month|Year|
+--------------------+---------------------+---------------+------------------+------------------+---------+------------------+---------+------------+-----------+----------+------------+------------+-----+----+
| 2009-02-03 08:25:00|  2009-02-03 08:33:39|            1.0|               1.6|        -73.992768|40.758325|         -73.99471|40.739723|        CASH|        6.9|       0.0|         0.0|         6.9|    2|2009|
| 2009-02-28 00:26:00|  2009-02-28 00:40:00|            5.0|              3.05|               0.0|      0.0|               0.0|      0.0|        CASH|      

In [33]:
new_complete_data.repartition(1).write.format("parquet").mode("append").save("gs://training-project-404110.appspot.com/FINAL_DATA/FINAL.parquet")

In [35]:
new_complete_data.write.format("parquet").mode("append").save("gs://training-project-404110.appspot.com/FINAL_MULTIPLE_DATA/FINAL.parquet")